In [45]:
# ---------------------------------------------------------------------------------
#       CNG 483 COMPUTER VISION PRACTICAL ASSIGNMENT 2
#       OMAR R.A. AMMAR     |   2456010
# ---------------------------------------------------------------------------------
# Importing required libraries
import matplotlib.pyplot as plt
import scipy
import numpy as np
from PIL import Image
from scipy import ndimage
import re
import os
from tensorflow import keras
import tensorflow as tf

# MLP with manual validation set
from keras.models import Sequential
from keras.layers import Dense

# getting the random seed for initializing the weights of the parameters
np.random.seed(1)



In [2]:
def relu(Z):
       return np.maximum(0, Z),Z

In [3]:
###################################################################################
#               Loading the datasets and separate them                           #
###################################################################################

In [4]:
# a function to import all the images from the file with its labels

def loadAllImages(folder):
    images = []
    for filename in os.listdir(folder):
        if filename.endswith(".jpg"):
            image = Image.open(os.path.join(folder, filename)).convert('RGB')
            image = image.resize((256, 256))
            npimage = np.array(image)
            if image is not None:
                # Using re.compile() + re.match() + re.groups()
                # Splitting text and number in string
                temp = re.compile("([a-zA-Z]+)([0-9]+)")
                res = temp.match(filename).groups()
                images.append((npimage,res[0]))
    
    return images




In [5]:


# a function that'll separate the dataset into 3 datasets
def separateDataSets(dataset):
    # first shuffle the dataset
    np.random.shuffle(dataset)
    
    datasetImages = []
    datasetLabels = []
    
    for image in dataset:
        datasetImages.append(image[0])
        datasetLabels.append(image[1])
    
    
    train_pct_index = int(0.6 * len(dataset))
    val_pct_index = int(train_pct_index+0.2*(len(dataset)))
        
    X_train,X_val, X_test = datasetImages[:train_pct_index], datasetImages[train_pct_index:val_pct_index], datasetImages[val_pct_index:]
    y_train,y_val ,y_test = datasetLabels[:train_pct_index], datasetLabels[train_pct_index:val_pct_index], datasetLabels[val_pct_index:]
    
    X_train1 = np.array(X_train).astype('float32')
    X_test1 = np.array(X_test).astype('float32')
    X_val1 = np.array(X_val).astype('float32')


    X_train1 /= 255
    X_test1 /= 255
    X_val1 /= 255

    
    return X_train1, y_train, X_val1, y_val ,X_test1, y_test

In [6]:
# the functions below is my try at building my model froms scratch 

In [7]:
###################################################################################
#               Initialization of the parameters                                  #
###################################################################################

In [8]:

def initializeParameters():
    # create the parameters dictionary
    parameters = {}
    L = len(layer_dims)  # number of layers in the network

    # initialize the weights in all the layers
    for l in range(1, L):
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l - 1]) * 0.01
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))

    return parameters


In [9]:

###################################################################################
#               Forward and Backward Propagation                                  #
###################################################################################


In [10]:
def forwardPropogation(X, W, b):
    # Z= WX+b
    Z = np.dot(W, X) + b

    cache = (X, W, b)

    return Z, cache


def linearActivation(A_prev, W, b):
    # I used relu as our activation function for forward propogation
    Z, linear_cache = forwardPropogation(A_prev, W, b)
    A, activation_cache = relu(Z)

    cache = (linear_cache, activation_cache)

    return A, cache


def L_model_forward(X, parameters):
    caches = []
    A = X
    L = len(parameters) // 2  # number of layers in the neural network

    # Implement [LINEAR -> RELU]*(L-1). Add "cache" to the "caches" list.
    for l in range(1, L):
        A_prev = A
        W = parameters['W' + str(l)]  # Get the parameters from the parameters dictionary
        b = parameters['b' + str(l)]
        A, cache = linearActivation(A_prev, W, b)
        caches.append(cache)

    W = parameters['W' + str(L)]
    b = parameters['b' + str(L)]
    AL, cache = linearActivation(A, W, b)
    caches.append(cache)

    return AL, caches


def linear_backward(dZ, cache):
    A_prev, W, b = cache
    m = A_prev.shape[1]

    dW = (1 / m) * np.dot(dZ, A_prev.T)
    db = (1 / m) * np.sum(dZ, axis=1, keepdims=True)
    dA_prev = np.dot(W.T, dZ)

    return dA_prev, dW, db


def linear_activation_backward(dA, cache):
    linear_cache, activation_cache = cache

    dZ = relu_backward(dA, activation_cache)
    dA_prev, dW, db = linear_backward(dZ, linear_cache)

    return dA_prev, dW, db


def update_parameters(parameters, grads, learning_rate):
    L = len(parameters) // 2  # number of layers in the neural network

    for l in range(L):
        parameters["W" + str(l + 1)] = parameters["W" + str(l + 1)] - learning_rate * grads["dW" + str(l + 1)]
        parameters["b" + str(l + 1)] = parameters["b" + str(l + 1)] - learning_rate * grads["db" + str(l + 1)]

    return parameters



In [11]:

###################################################################################
#               Computing the cost                                               #
###################################################################################



In [12]:
def compute_cost(AL, Y):
    m = Y.shape[1]
    # Compute loss from aL and y.

    cost = (-1 / m) * (np.dot(Y, np.log(AL).T) + np.dot((1 - Y), np.log(1 - AL).T))  #

    cost = np.squeeze(cost)  # To make sure cost's shape is what we expect
    return cost


In [13]:

###################################################################################
#               Building the model                                                #
###################################################################################



In [14]:
def L_layer_model(X, Y, layers_dims, learning_rate=0.0075, num_iterations=3000, print_cost=False, plot_graph=False):
    costs = []  # keep track of cost

    # Parameters initialization.
    parameters = initializeParameters()

    # Loop (gradient descent)
    for i in range(0, num_iterations):
        # forward pass
        AL, caches = L_model_forward(X, parameters)

        # Compute cost.
        cost = compute_cost(AL, Y)

        # Backward pass.
        grads = L_model_backward(AL, Y, caches)

        # Update parameters after every pass.
        parameters = update_parameters(parameters, grads, learning_rate)

        # Print the cost every 100 training example
        if print_cost and i % 100 == 0:
            print("Cost after iteration %i: %f" % (i, cost))
            costs.append(cost)

    # plot the cost
    if plot_graph:
        plt.plot(np.squeeze(costs))
        plt.ylabel('cost')
        plt.xlabel('iterations (per tens)')
        plt.title("Learning rate =" + str(learning_rate))
        plt.show()

    return parameters



In [15]:

###################################################################################
#         Model's predictions vs the actual labels                               #
###################################################################################

def predict(images, labels, parameters):
    pass



In [16]:

# load all the images from the folder
# CHANGE THE NEXT LINE WITH YOUR OWN PATH
dataset = loadAllImages("/Users/omarammar/PycharmProjects/ComVis2/Dataset")   


   

In [17]:
np.random.shuffle(dataset)

In [18]:
print('the size of every image in the dataset: ',dataset[0][0].shape)
print('number of examples in the dataset: ',len(dataset))

the size of every image in the dataset:  (256, 256, 3)
number of examples in the dataset:  907


In [19]:
X_train, y_train, X_val, y_val,X_test, y_test = separateDataSets(dataset) 


In [20]:
# make sure of the training and testing data shapes
print('Number of images in the training set: '+ str(X_train.shape[0]))
print('Number of images in the validation set: '+ str(X_val.shape[0]))
print('Number of images in the test set: '+ str(X_test.shape[0]))
print('size of images in the training set',X_train[0].shape)
print('The size of the training set: ',X_train.shape)
print('The size of the test set: ',X_test.shape)


Number of images in the training set: 544
Number of images in the validation set: 181
Number of images in the test set: 182
size of images in the training set (256, 256, 3)
The size of the training set:  (544, 256, 256, 3)
The size of the test set:  (182, 256, 256, 3)


In [21]:
# one-hot coded labels 
y_train_new = []
for y in y_train:
    if y == 'cloudy':
        y_train_new.append([0,0,1])
    elif y == 'sunrise':
        y_train_new.append([0,1,0])
    else:
        y_train_new.append([1,0,0])


In [22]:
y_train = np.array(y_train_new)

In [23]:
# one-hot coded labels 
y_test_new = []
for y in y_test:
    if y == 'cloudy':
        y_test_new.append([0,0,1])
    elif y == 'sunrise':
        y_test_new.append([0,1,0])
    else:
        y_test_new.append([1,0,0])


In [24]:
y_test = np.array(y_test_new)

In [25]:
# one-hot coded labels 
y_val_new = []
for y in y_val:
    if y == 'cloudy':
        y_val_new.append([0,0,1])
    elif y == 'sunrise':
        y_val_new.append([0,1,0])
    else:
        y_val_new.append([1,0,0])


In [26]:
y_val = np.array(y_val_new)

In [27]:
# testing with the first setting 
# 2 layer model 
# input layer is a flattening layer
# layer 1 has 256 nodes and uses relu as activation
# layer 2 has 128 nodes and uses relu as activation
# output layer uses softmax and has 3 nodes

# create model
model1 = Sequential()
model1.add(keras.layers.Flatten())
model1.add(Dense(256, activation='relu'))
model1.add(Dense(128, activation='relu'))
model1.add(Dense(3, activation='softmax'))
# Compile model
model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# Fit the model
model1.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, batch_size=10)


Epoch 1/50


2022-06-17 11:38:20.892232: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


55/55 [==============================] - 7s 111ms/step - loss: 19.0431 - accuracy: 0.7040 - val_loss: 33.5680 - val_accuracy: 0.6133
Epoch 2/50
55/55 [==============================] - 7s 113ms/step - loss: 8.8729 - accuracy: 0.7923 - val_loss: 11.5191 - val_accuracy: 0.6409
Epoch 3/50
55/55 [==============================] - 6s 113ms/step - loss: 12.0969 - accuracy: 0.7463 - val_loss: 4.3414 - val_accuracy: 0.8177
Epoch 4/50
55/55 [==============================] - 6s 111ms/step - loss: 1.8617 - accuracy: 0.8419 - val_loss: 1.8508 - val_accuracy: 0.8287
Epoch 5/50
55/55 [==============================] - 6s 118ms/step - loss: 1.8381 - accuracy: 0.7978 - val_loss: 2.6416 - val_accuracy: 0.8287
Epoch 6/50
55/55 [==============================] - 6s 113ms/step - loss: 1.4331 - accuracy: 0.8548 - val_loss: 2.5819 - val_accuracy: 0.8011
Epoch 7/50
55/55 [==============================] - 6s 109ms/step - loss: 0.9565 - accuracy: 0.8879 - val_loss: 2.1558 - val_accuracy: 0.7514
Epoch 8/50
55

In [28]:
'''
    The previous model got a validation accuracy of 0.8343 , and training accuracy of 0.9320 
    On the other hand the loss was fairly low, for training loss it was 0.2126 and validation loss of 0.8349

'''

'\n    The previous model got a validation accuracy of 0.8885 , and training accuracy of 0.9210 \n    On the other hand the loss was fairly low, for training loss it was 0.2136 and validation loss of 0.7528\n\n'

In [29]:
# testing with the first setting 
# 3 layer model 
# input layer is a flattening layer
# layer 1 has 256 nodes and uses relu as activation
# layer 2 has 128 nodes and uses relu as activation
# layer 3 has 64 nodes and uses relu as activation
# output layer uses softmax and has 3 nodes

# create model
model2 = Sequential()
model2.add(keras.layers.Flatten())
model2.add(Dense(256, activation='relu'))
model2.add(Dense(128, activation='relu'))
model2.add(Dense(64, activation='relu'))
model2.add(Dense(3, activation='softmax'))
# Compile model
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# Fit the model
model2.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, batch_size=10)


Epoch 1/50
55/55 [==============================] - 7s 110ms/step - loss: 13.1207 - accuracy: 0.6360 - val_loss: 15.8457 - val_accuracy: 0.5304
Epoch 2/50
55/55 [==============================] - 6s 112ms/step - loss: 4.3093 - accuracy: 0.7831 - val_loss: 2.7252 - val_accuracy: 0.7956
Epoch 3/50
55/55 [==============================] - 6s 117ms/step - loss: 2.6495 - accuracy: 0.8143 - val_loss: 4.1522 - val_accuracy: 0.7017
Epoch 4/50
55/55 [==============================] - 6s 117ms/step - loss: 2.7814 - accuracy: 0.7923 - val_loss: 5.7406 - val_accuracy: 0.7293
Epoch 5/50
55/55 [==============================] - 7s 121ms/step - loss: 1.4916 - accuracy: 0.8511 - val_loss: 1.6266 - val_accuracy: 0.7845
Epoch 6/50
55/55 [==============================] - 6s 114ms/step - loss: 1.8274 - accuracy: 0.8272 - val_loss: 6.8332 - val_accuracy: 0.7182
Epoch 7/50
55/55 [==============================] - 6s 112ms/step - loss: 1.9654 - accuracy: 0.8033 - val_loss: 1.5402 - val_accuracy: 0.8508
Epoc

In [30]:
'''
    The previous model got a validation accuracy of 0.8729 , and training accuracy of 0.9669 
    On the other hand the loss was fairly high, for training loss it was 0.0772 and validation loss of 0.8729

'''

'\n    The previous model got a validation accuracy of 0.8177 , and training accuracy of 0.9320 \n    On the other hand the loss was fairly high, for training loss it was 0.2562 and validation loss of 0.9942\n\n'

In [31]:
# testing with the first setting 
# 3 layer model 
# input layer is a flattening layer
# layer 1 has 1024 nodes and uses relu as activation
# layer 2 has 512 nodes and uses relu as activation
# layer 3 has 256 nodes and uses relu as activation
# output layer uses softmax and has 3 nodes

In [32]:
# running this model would take a lot of time and power from the machine, don't run it unless u willing to wait for 2 hours
# create model
model3 = Sequential()
model3.add(keras.layers.Flatten())
model3.add(Dense(1024, activation='relu'))
model3.add(Dense(512, activation='relu'))
model3.add(Dense(256, activation='relu'))
model3.add(Dense(3, activation='softmax'))
# Compile model
model3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# Fit the model
model3.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, batch_size=10)


Epoch 1/50
55/55 [==============================] - 153s 3s/step - loss: 32.4843 - accuracy: 0.6562 - val_loss: 4.3592 - val_accuracy: 0.7624
Epoch 2/50
55/55 [==============================] - 238s 4s/step - loss: 4.2036 - accuracy: 0.7555 - val_loss: 10.2568 - val_accuracy: 0.6796
Epoch 3/50
55/55 [==============================] - 206s 4s/step - loss: 2.1639 - accuracy: 0.8088 - val_loss: 2.5003 - val_accuracy: 0.7072
Epoch 4/50
55/55 [==============================] - 154s 3s/step - loss: 1.4345 - accuracy: 0.8051 - val_loss: 2.8899 - val_accuracy: 0.6851
Epoch 5/50
55/55 [==============================] - 141s 3s/step - loss: 1.4721 - accuracy: 0.8033 - val_loss: 1.7736 - val_accuracy: 0.7127
Epoch 6/50
55/55 [==============================] - 134s 2s/step - loss: 0.6616 - accuracy: 0.8529 - val_loss: 1.0097 - val_accuracy: 0.8619
Epoch 7/50
55/55 [==============================] - 125s 2s/step - loss: 0.5714 - accuracy: 0.8658 - val_loss: 1.3543 - val_accuracy: 0.8122
Epoch 8/50


In [33]:
'''
spike in loss at the beginning of the training
training took a very long time. because of how complex the nn got
 The previous model got a validation accuracy of 0.8840 , and training accuracy of 0.9467 
    On the other hand the loss was fairly high, for training loss it was 0.1186 and validation loss of 1.4840
    although we added more layers and more nodes it still offered worse results than our first model with only
    2 layers , I belive this was because of the model overfitting.
'''

'\nspike in loss at the beginning of the training\ntraining took a very long time. because of how complex the nn got\n The previous model got a validation accuracy of 0.8674 , and training accuracy of 0.9007 \n    On the other hand the loss was fairly high, for training loss it was 0.2638 and validation loss of 1.3051\n    although we added more layers and more nodes it still offered worse results than our first model with only\n    2 layers , I belive this was because of the model overfitting.\n'

In [34]:
# testing with the first setting 
# 4 layer model 
# input layer is a flattening layer
# layer 1 has 512 nodes and uses relu as activation
# layer 2 has 256 nodes and uses relu as activation
# layer 3 has 128 nodes and uses relu as activation
# layer 4 has 64 nodes and uses relu as activation
# output layer uses softmax and has 3 nodes


# create model
model4 = Sequential()
model4.add(keras.layers.Flatten())
model4.add(Dense(512, activation='relu'))
model4.add(Dense(256, activation='relu'))
model4.add(Dense(128, activation='relu'))
model4.add(Dense(64, activation='relu'))
model4.add(Dense(3, activation='softmax'))
# Compile model
model4.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# Fit the model
model4.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, batch_size=100)

Epoch 1/50
6/6 [==============================] - 14s 1s/step - loss: 70.0076 - accuracy: 0.4118 - val_loss: 82.2905 - val_accuracy: 0.3370
Epoch 2/50
6/6 [==============================] - 5s 811ms/step - loss: 33.5356 - accuracy: 0.5092 - val_loss: 17.7801 - val_accuracy: 0.5912
Epoch 3/50
6/6 [==============================] - 4s 703ms/step - loss: 9.9150 - accuracy: 0.6562 - val_loss: 16.8369 - val_accuracy: 0.6685
Epoch 4/50
6/6 [==============================] - 4s 619ms/step - loss: 8.4270 - accuracy: 0.7206 - val_loss: 3.6707 - val_accuracy: 0.7127
Epoch 5/50
6/6 [==============================] - 5s 793ms/step - loss: 4.5277 - accuracy: 0.7500 - val_loss: 3.0979 - val_accuracy: 0.7790
Epoch 6/50
6/6 [==============================] - 4s 662ms/step - loss: 2.6892 - accuracy: 0.8088 - val_loss: 4.2910 - val_accuracy: 0.7403
Epoch 7/50
6/6 [==============================] - 4s 613ms/step - loss: 2.5417 - accuracy: 0.7923 - val_loss: 4.7801 - val_accuracy: 0.7624
Epoch 8/50
6/6 [=

In [35]:
'''
    spike in the loss at the beginning of the training
    The previous model got a validation accuracy of 0.7680 , and training accuracy of 0.9688
    On the other hand the loss was fairly high, for training loss it was 0.0966 and validation loss of 1.813
    There is a big gab between the training and the validation scores which could tell us that the model is overfitting

'''

'\n    spike in the loss at the beginning of the training\n    The previous model got a validation accuracy of 0.8619 , and training accuracy of 0.9062\n    On the other hand the loss was fairly high, for training loss it was 0.5065 and validation loss of 1.5096\n\n'

In [36]:
###########                  QUESTIONS               #################

# 1) Discussion on the effects of the number of layers.
'''
The number of layers in a model is referred to as its depth. 
Increasing the depth increases the capacity of the model. Training deep models, e.g. those with many hidden layers, 
can be computationally more efficient than training a single layer network with a vast number of nodes

We can also see from the models I trained before that with increasing the number of layers the training time
increases as the complexity of the model gets bigger and harder to compute.

increasing the number of layers doesn't always mean having a better accuracy and loss as you can see in the comparsion 
between my 2-layer model(First trained model) and my 4-layer model (4th trained model)
'''

# 2) Rationale behind your choices of hyper-parameters like number of layers, number of epochs, layer sizes etc
'''
number of layers: 
    I tested my learning model with 3 different sizes of layers ( 2, 3, 4) layered models to compare the accuracy of 
    each model with more layers, check the effect of number of layers in the previous question for more details.
    
number of epochs: 
    I used 50 epoches for running all my models to be able to compare between all my models.

layer sizes: 
    I used all my layer sizes to be multiples of 2 , because images themselves being put as an input are a multiple
    of 2 ( all images were resized to 256)
'''



'\nnumber of layers: \n    I tested my learning model with 3 different sizes of layers ( 2, 3, 4) layered models to compare the accuracy of \n    each model with more layers, check the effect of number of layers in the previous question for more details.\n    \nnumber of epochs: \n    I used 50 epoches for running all my models to be able to compare between all my models.\n\nlayer sizes: \n    I used all my layer sizes to be multiples of 2 , because images themselves being put as an input are a multiple\n    of 2 ( all images were resized to 256)\n'

In [37]:
####################              Model testing with the best configuration          ##################

In [38]:
# we saw that the best model with the highest validation accuracy was the third model with 4 hidden layers , thus we are
# going to run the test with this configuration

In [43]:
print("Evaluate on test data")
results = model3.evaluate(X_test, y_test, batch_size=10)
print("test loss, test acc:", results)

Evaluate on test data
19/19 [==============================] - 4s 95ms/step - loss: 0.6698 - accuracy: 0.8736
test loss, test acc: [0.6697514057159424, 0.8736263513565063]


In [44]:
# Evaluation of testing with the best configuration 
'''
    From testing model 1 , we got a fairly good accuracy of 0.8736 , I believe the accuracy could be better by 
    varying the dataset , as we randomized the selection of the images in the dataset we could get better results
    
'''

'\n    From testing model 1 , we got a fairly good accuracy of 0.8736 , I believe the accuracy could be better by \n    varying the dataset , as we randomized the selection of the images in the dataset we could get better results\n    \n'